In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import datetime as dt 
import yfinance as yf

from sklearn.metrics import mean_squared_error , mean_absolute_error , explained_variance_score , r2_score
from sklearn.metrics import confusion_matrix , classification_report , accuracy_score , roc_auc_score , roc_curve
from sklearn.metrics import mean_poisson_deviance , mean_gamma_deviance , accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , BatchNormalization

from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [5]:
df = yf.download('BTC-USD' ,start='2014-09-17' , end='2024-01-03')

[*********************100%***********************]  1 of 1 completed


In [6]:
df.to_csv('BTC-USD.csv')

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-12-29,42614.644531,43124.324219,41424.062500,42099.402344,42099.402344,26000021055
2023-12-30,42091.753906,42584.125000,41556.226562,42156.902344,42156.902344,16013925945
2023-12-31,42152.097656,42860.937500,41998.253906,42265.187500,42265.187500,16397498810


In [8]:
df.shape

(3395, 6)

In [9]:
print("total number of days in the dataset : ", df.shape[0])
print("total number of features in the dataset : ", df.shape[1])

total number of days in the dataset :  3395
total number of features in the dataset :  6
